In [13]:
import qiime2
from qiime2.plugins import (
    cutadapt, demux, dada2, feature_table, metadata,
    greengenes2, taxa, feature_classifier,
    vsearch
)

from qiime2 import Artifact, Metadata
import os

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

path = {
    "art" : "../data/artifacts/mock/",
    "vis" : "../visualizations/mock/",
    "res" : "../data/resources/"
 }

for filepath in path.values():
    os.makedirs(filepath, exist_ok=True)

In [9]:
# create mock metadata
metadata_df = pd.DataFrame({
    'sampleid': ['2476', '2477'], 'mock' : [0, 1]}).set_index('sampleid')

metadata_q2 = qiime2.Metadata(metadata_df)

In [10]:
# import raw data with manifest
raw_seqs = qiime2.Artifact.import_data('SampleData[PairedEndSequencesWithQuality]',
                                       '../data/manifest-mock.tsv', view_type='PairedEndFastqManifestPhred33V2')

# 1. Quality control 
## 1.1. DADA2 
Clearing sequences from artifacts of sequencing

In [11]:
quality_vis = demux.visualizers.summarize(raw_seqs)
quality_vis.visualization.save(path["vis"] + "quality-plot.qzv")

'../visualizations/mock/quality-plot.qzv'

<Figure size 640x480 with 0 Axes>

## Interpretation

Desired length of forward + reverse primers should cover full V3-V4 hypervariable region (~465 bp). DADA2 expects >= 12 bp of overlap. The quality of reverse read is not great (quality score of 20 means that 1% of pairs is erroneous). However, this should not hurt species classification much.

In [14]:
qc_reads = dada2.methods.denoise_paired(
    raw_seqs, trunc_len_f=260, trunc_len_r=230, n_threads=32,
    min_fold_parent_over_abundance=4
)

qc_reads.denoising_stats.save(path["art"] + "denoise-stats.qza")
qc_reads.table.save(path["art"] + "feature-table.qza")
qc_reads.representative_sequences.save(path["art"] + "rep-seqs.qza")


metadata.visualizers.tabulate(input=qc_reads.denoising_stats.view(Metadata)).visualization.save(path["vis"] + "denoise-stats.qzv")
feature_table.visualizers.summarize(qc_reads.table).visualization.save(path["vis"] + "feature-table.qzv")

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada.R --input_directory /tmp/tmpq9ckuw6g/forward --input_directory_reverse /tmp/tmpq9ckuw6g/reverse --output_path /tmp/tmpq9ckuw6g/output.tsv.biom --output_track /tmp/tmpq9ckuw6g/track.tsv --filtered_directory /tmp/tmpq9ckuw6g/filt_f --filtered_directory_reverse /tmp/tmpq9ckuw6g/filt_r --truncation_length 260 --truncation_length_reverse 230 --trim_left 0 --trim_left_reverse 0 --max_expected_errors 2.0 --max_expected_errors_reverse 2.0 --truncation_quality_score 2 --min_overlap 12 --pooling_method independent --chimera_method consensus --min_parental_fold 4 --allow_one_off False --num_threads 32 --learn_min_reads 1000000



Warning message:
package ‘optparse’ was built under R version 4.2.3 
Loading required package: Rcpp


R version 4.2.2 (2022-10-31) 
DADA2: 1.26.0 / Rcpp: 1.0.12 / RcppParallel: 5.1.6 
2) Filtering ..
3) Learning Error Rates
13288600 total bases in 51110 reads from 2 samples will be used for learning the error rates.
11755300 total bases in 51110 reads from 2 samples will be used for learning the error rates.
3) Denoise samples ..
..
5) Remove chimeras (method = consensus)
6) Report read numbers through the pipeline
7) Write output


'../visualizations/mock/feature-table.qzv'

## Interpretation 

DADA2 again (3rd time) discards too many reads, which is a sign of a poor quality of sequencing. Only 35% of reads pass the initial filtering. Overall, 29% of reads passed the quality control step. 

# 2. Taxonomical classification - bacterial 16S

In [15]:
# load preprocessed data
table = Artifact.load(path["art"] + "feature-table.qza")
rep_seqs = Artifact.load(path["art"] + "rep-seqs.qza")

## Greengenes 2

In [16]:
gg2_mapped = vsearch.methods.cluster_features_closed_reference(sequences=rep_seqs, table=table,
                                                               reference_sequences=Artifact.load(path["res"] + "2022.10.backbone.full-length.fna.qza"),
                                                               perc_identity=0.99, threads=8)

gg2_mapped.clustered_table.save(path["art"] + "feature-table-gg2.qza")
gg2_mapped.clustered_sequences.save(path["art"] + "rep-seqs-gg2.qza")

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --usearch_global /tmp/tmp87l7vgpb --id 0.99 --db /tmp/qiime2/vbezshapkin/data/a53d9300-5c5c-4774-a2e8-a5e23904f1ae/data/dna-sequences.fasta --uc /tmp/tmptuqmd1zu --strand plus --qmask none --notmatched /tmp/tmpfuf3spon --threads 8 --minseqlength 1 --fasta_width 0



vsearch v2.22.1_linux_x86_64, 2011.2GB RAM, 256 cores
https://github.com/torognes/vsearch

Reading file /tmp/qiime2/vbezshapkin/data/a53d9300-5c5c-4774-a2e8-a5e23904f1ae/data/dna-sequences.fasta 100%
494630940 nt in 331269 seqs, min 416, max 4563, avg 1493
Masking 100%
Counting k-mers 100%
Creating k-mer index 100%
Searching 100%
Matching unique query sequences: 465 of 479 (97.08%)
vsearch v2.22.1_linux_x86_64, 2011.2GB RAM, 256 cores
https://github.com/torognes/vsearch

Reading file /tmp/tmpfuf3spon 100%
6042 nt in 14 seqs, min 277, max 465, avg 432
Getting sizes 100%
Sorting 100%
Median abundance: 3
Writing output 100%


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --sortbysize /tmp/tmpfuf3spon --xsize --output /tmp/q2-DNAFASTAFormat-gl0ry9bc --minseqlength 1 --fasta_width 0



'../data/artifacts/mock/rep-seqs-gg2.qza'

In [17]:
tax = greengenes2.methods.taxonomy_from_table(table = gg2_mapped.clustered_table,
                                              reference_taxonomy = Artifact.load(path["res"] + "gg2.2022.10.taxonomy.asv.nwk.qza"))

In [18]:
tax.classification.save(path["art"] + "tax-gg2.qza")

'../data/artifacts/mock/tax-gg2.qza'

In [19]:
vis = taxa.visualizers.barplot(table = gg2_mapped.clustered_table,
                               taxonomy = Artifact.load(path["art"] + "tax-gg2.qza"),
                               metadata = metadata_q2)
vis.visualization.save(path["vis"] + "gg2-barplot.qzv")

'../visualizations/mock/gg2-barplot.qzv'

In [20]:
# percentage of unmapped sequences

unmapped_perc = gg2_mapped.unmatched_sequences.view(pd.Series).shape[0] / rep_seqs.view(pd.Series).shape[0]
print('Percentage of unmapped sequences: {:.2f}%'.format(unmapped_perc*100))

Percentage of unmapped sequences: 2.92%
